In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

In [2]:
raw_df = pd.read_excel("../data/FINAL_NUM_1101_2.xlsx")
pd.set_option('display.max_columns',35)
raw_df

,DONG_CODE,SI_DO,GU,DONG,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,BUS_NUM,SUBWAY_NUM,SAFE_DLVR_NUM,DPTM_NUM,ANI_HSPT_NUM,PHARM_NUM,LEISURE_NUM,KIDS_NUM,HOSPITAL_NUM,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,STD_DAY,PARK_NUM,RETAIL_NUM
0,1147010100,서울특별시,양천구,신정동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,5
1,1150010100,서울특별시,강서구,염창동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,2
2,1150010400,서울특별시,강서구,가양동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,3
3,1150010200,서울특별시,강서구,등촌동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,1
4,1150010300,서울특별시,강서구,화곡동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,1123010700,서울특별시,동대문구,청량리동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,1
463,1123010800,서울특별시,동대문구,회기동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,2
464,1123010900,서울특별시,동대문구,휘경동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,1
465,1123011000,서울특별시,동대문구,이문동,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022-11-01,0,4


In [3]:
retail = raw_df[['RETAIL_NUM']]
retail

,RETAIL_NUM
0,5
1,2
2,3
3,1
4,7
...,...
462,1
463,2
464,1
465,4


In [4]:
def dong_change(df):
    sample_df = df
    sample_df.drop('STD_DAY',axis=1,inplace=True)

    change_df = pd.read_excel('행정구역분류.xlsx',header=1)

    seoul_df = change_df[change_df['시도']=='서울특별시']
    seoul_df = seoul_df[seoul_df['행정구역명'].str[-1] =='동']
    seoul_df = seoul_df[['시군구','행정구역명','법정동','법정동코드','행정기관코드','법정동 관할구역\n분할여부']]
    seoul_df.columns = ['GU','DONG','법정동','DONG_CODE','행정기관코드','분할여부']
    non_div = seoul_df[seoul_df['분할여부'].isnull()][['GU','DONG','DONG_CODE','행정기관코드']]

    remove_du = non_div.drop_duplicates(['행정기관코드'])

    tmp_sample_df = sample_df.drop(['GU','DONG'],axis=1)
    con_df1 = pd.merge(tmp_sample_df,non_div,on="DONG_CODE")

    non_div_result = con_df1.groupby('행정기관코드').sum()
    non_div_result = non_div_result.drop(['DONG_CODE'],axis=1)

    non_div_result = pd.merge(non_div_result, remove_du,right_on='행정기관코드',left_index=True)
    non_div_result.drop('DONG_CODE',inplace=True,axis=1)

    non_div_result.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

    div_df = seoul_df[seoul_df['분할여부']=='분할연계'][['GU','DONG','DONG_CODE','행정기관코드']]

    div_dong_count = div_df.groupby('DONG_CODE').count()[['행정기관코드']]
    div_dong_count.columns = ['DONG_COUNT']

    con_df2 = pd.merge(sample_df, div_dong_count,left_on='DONG_CODE',right_index=True)
    con_df2.set_index('DONG_CODE',inplace=True)

    con_df2 = con_df2.iloc[:,3:]
    cal_df = con_df2.div(con_df2['DONG_COUNT'], axis=0) 
    div_result_df = pd.merge(div_df,cal_df,left_on='DONG_CODE',right_index=True)
    div_result_df = div_result_df.drop(['DONG_CODE','DONG_COUNT'],axis=1)
    div_result_df.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

    non_div_result = non_div_result[div_result_df.columns]
    result_df = pd.concat([non_div_result, div_result_df],axis=0)
    result_sum = result_df.groupby('DONG_CODE').sum()
    
    remove_result_du =result_df.drop_duplicates(['DONG_CODE'])[['GU','DONG','DONG_CODE']]
    result_df2 = pd.merge(result_sum, remove_result_du,right_on='DONG_CODE',left_index=True)

    return result_df2

In [5]:
change_df = dong_change(raw_df)
change_df

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,BUS_NUM,SUBWAY_NUM,SAFE_DLVR_NUM,DPTM_NUM,ANI_HSPT_NUM,PHARM_NUM,LEISURE_NUM,KIDS_NUM,HOSPITAL_NUM,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,RETAIL_NUM,GU,DONG,DONG_CODE
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,청운효자동,1111051500
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,종로구,사직동,1111053000
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,삼청동,1111054000
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,부암동,1111055000
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,평창동,1111056000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666667,강동구,성내2동,1174065000
763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666667,강동구,성내3동,1174066000
764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,강동구,길동,1174068500
765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,강동구,둔촌1동,1174069000


In [6]:
def add_bus_data(df):
    df.drop('BUS_NUM',axis=1,inplace=True)
    copy_df = df.copy()
    copy_df['GU_DONG'] = copy_df['GU'] + copy_df['DONG']
    
    bus_df = pd.read_csv('동별_버스정보_수정.csv')
    merge_df = pd.merge(copy_df, bus_df)
    merge_df.drop('GU_DONG',axis=1,inplace=True)
    
    return merge_df

In [7]:
final_df = add_bus_data(change_df)
final_df

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,SUBWAY_NUM,SAFE_DLVR_NUM,DPTM_NUM,ANI_HSPT_NUM,PHARM_NUM,LEISURE_NUM,KIDS_NUM,HOSPITAL_NUM,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,RETAIL_NUM,GU,DONG,DONG_CODE,BUS_CNT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,청운효자동,1111051500,14.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,종로구,사직동,1111053000,16.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,삼청동,1111054000,11.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,부암동,1111055000,9.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,종로구,평창동,1111056000,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666667,강동구,성내2동,1174065000,11.0
422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666667,강동구,성내3동,1174066000,9.0
423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,강동구,길동,1174068500,28.0
424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,강동구,둔촌1동,1174069000,4.0


In [8]:
final_df[['RETAIL_NUM']].describe()

,RETAIL_NUM
count,426.000000
mean,1.497653
std,1.452043
min,0.000000
25%,0.666667
50%,1.200000
75%,2.000000
max,14.000000


In [9]:
final_df.to_csv('대형마트_행정동_변환_1102.csv',mode='w')

In [ ]:
retail = pd.read_csv('대형마트_행정동_변환_1102.csv')
retail

In [ ]:
retail = retail[['RETAIL_NUM']]

In [ ]:
tmp_df = pd.read_csv('성형외과제거_및_버스수정_행정동_변환_1101.csv')

In [ ]:
tmp_df['RETAIL_NUM'] = retail
tmp_df

In [ ]:
del(tmp_df['Unnamed: 0'])

In [ ]:
tmp_df.to_csv('버스,대형마트,병원_수정_1102.csv',mode='w')

In [10]:
tmp_df = pd.read_csv('버스,대형마트,병원_수정_1102.csv')
tmp_df

,Unnamed: 0,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,POLICE_NUM,PHARM_NUM,NOISE_VIBRATION_NUM,MID_SCH_NUM,MC_NUM,LEISURE_NUM,KINDER_NUM,KIDS_NUM,HIGH_SCH_NUM,GYM_NUM,GOLF_NUM,FIRE_NUM,ELE_SCH_NUM,DPTM_NUM,CON_NUM,CHILD_MED_NUM,CCTV_NUM,CAR_SHR_NUM,CAFE_NUM,BIKE_NUM,ANI_HSPT_NUM,ACADEMY_NUM,HOSPITAL_NUM,GU,DONG,DONG_CODE,BUS_CNT,RETAIL_NUM
0,0,0.333333,0.333333,0.0,1.000000,2.000000,4.333333,18816.000000,2.000000,0.0,1.000000,0.000000,0.000000,3.0,0.0,0.000000,1.0,2.000000,0.0,4.000000,7.333333,56.666667,1.000000,39.666667,5.666667,2.000000,18.000000,11.666667,종로구,청운효자동,1111051500,14.0,0.000000
1,1,1.333333,7.333333,0.0,0.000000,3.000000,17.333333,22848.000000,0.000000,0.0,1.000000,10.000000,0.000000,0.0,18.0,3.000000,0.0,2.000000,0.0,20.000000,10.333333,41.666667,5.000000,100.666667,11.666667,1.000000,19.000000,74.666667,종로구,사직동,1111053000,16.0,1.000000
2,2,1.000000,1.000000,1.0,0.000000,1.000000,2.000000,14112.000000,1.000000,0.0,0.000000,10.000000,0.000000,1.0,3.0,0.000000,0.0,0.000000,0.0,2.000000,1.000000,22.000000,1.000000,52.000000,7.000000,0.000000,2.000000,7.000000,종로구,삼청동,1111054000,11.0,0.000000
3,3,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,6048.000000,1.000000,0.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,1.0,4.000000,0.0,8.000000,5.000000,41.000000,5.000000,30.000000,5.000000,3.000000,24.000000,4.000000,종로구,부암동,1111055000,9.0,0.000000
4,4,0.000000,1.000000,0.0,0.000000,1.000000,5.000000,4032.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,5.0,5.000000,0.0,0.000000,0.0,6.000000,10.000000,21.000000,0.000000,36.000000,7.000000,0.000000,28.000000,14.000000,종로구,평창동,1111056000,37.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,421,0.333333,1.333333,0.0,0.666667,0.333333,12.666667,1442.333333,0.333333,0.0,1.666667,16.666667,0.666667,0.0,10.0,4.666667,1.0,1.333333,0.0,38.666667,4.000000,52.000000,6.666667,47.666667,6.000000,2.666667,63.333333,59.333333,강동구,성내2동,1174065000,11.0,1.666667
422,422,0.333333,1.333333,0.0,0.666667,0.333333,12.666667,1442.333333,0.333333,0.0,1.666667,16.666667,0.666667,0.0,10.0,4.666667,1.0,1.333333,0.0,38.666667,4.000000,52.000000,6.666667,47.666667,6.000000,2.666667,63.333333,59.333333,강동구,성내3동,1174066000,9.0,1.666667
423,423,1.000000,2.000000,0.0,1.000000,0.000000,35.000000,4327.000000,1.000000,0.0,3.000000,20.000000,1.000000,0.0,18.0,9.000000,1.0,6.000000,0.0,76.000000,10.000000,119.000000,10.000000,63.000000,9.000000,8.000000,100.000000,120.000000,강동구,길동,1174068500,28.0,4.000000
424,424,0.500000,0.000000,0.0,0.000000,1.000000,10.000000,2163.500000,1.500000,0.5,0.000000,0.000000,0.000000,1.0,4.0,3.000000,0.0,2.000000,0.0,17.000000,0.000000,32.500000,2.000000,17.500000,5.500000,1.500000,38.000000,14.500000,강동구,둔촌1동,1174069000,4.0,0.000000


In [12]:
mz = pd.read_csv('../data/MZ세대인구수.csv')
mz

,Unnamed: 0,GU,DONG,20~24세,25~29세,30~34세,35~39세,인구수
0,0,종로구,사직동,473,753,802,721,2749
1,1,종로구,삼청동,153,178,161,192,684
2,2,종로구,부암동,630,673,644,638,2585
3,3,종로구,평창동,1108,1246,1050,1193,4597
4,4,종로구,무악동,469,442,384,527,1822
...,...,...,...,...,...,...,...,...
421,421,강동구,암사1동,2025,2744,2478,2513,9760
422,422,강동구,천호2동,1787,3193,3342,2883,11205
423,423,강동구,길동,2661,4368,3994,3463,14486
424,424,강동구,상일1동,1937,1939,2363,3536,9775


In [13]:
tmp_df[['GU','DONG']]

,GU,DONG
0,종로구,청운효자동
1,종로구,사직동
2,종로구,삼청동
3,종로구,부암동
4,종로구,평창동
...,...,...
421,강동구,성내2동
422,강동구,성내3동
423,강동구,길동
424,강동구,둔촌1동
